In [ ]:
import sys
print(sys.executable)

## 1. First we import necessary Libaries

In [ ]:
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import re
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

### 2. Load Movies Data

In [ ]:
filename = "../datasets/movies.csv"

In [ ]:
movies = pd.read_csv(filename)

In [ ]:
movies.sample(5)

### 2.1 Reading Movie Release Year

In [ ]:
def title_to_release_year(s):
    m = re.findall(r"\(([0-9]+)\)", s)
    
    # https://www.guru99.com/python-regular-expressions-complete-tutorial.html
    if m is None or len(m) <= 0:
        return None        
    return m[-1]

movies['year']  = movies['title'].apply(title_to_release_year)
movies['year'].fillna("1900", inplace = True)

### 2.2 Create Genres 

In [ ]:
movies['genres_list'] = movies.apply(lambda row: row.genres.split('|'), axis=1)

# https://stackoverflow.com/a/58114603
genres_encoder = MultiLabelBinarizer()
transformed = genres_encoder.fit_transform(movies['genres_list'])
print(len(genres_encoder.classes_), genres_encoder.classes_)

ohe_df = pd.DataFrame(transformed)
data = pd.concat([movies, ohe_df], axis=1)

data.sample(5)

### 2.3 Get item to item mapping 

In [ ]:
item_id_mapping = pd.read_csv("../datasets/data/item_id_mapping.txt", sep='\t', index_col=False)
item_id_mapping = item_id_mapping.sort_values(by=['new_id'])
item_id_mapping = item_id_mapping.rename(columns={'original_id':'movieId'})
item_id_mapping.head(5)

In [ ]:
item_id_mapping.shape

In [ ]:
movies_data=item_id_mapping.merge(data, on = 'movieId', how = 'inner')
movies_data.head()

In [ ]:
movies_data.shape

In [ ]:
columns = ['movieId', 'year'] + list(data.columns[-20:])
# print(columns)
temp_data = movies_data.loc[:][columns]

temp_data.head()

In [ ]:
temp_data.to_csv("../datasets/data/movies_features.txt", index=False)

In [ ]:
np.array(temp_data.head())

In [ ]:
new_temp_data = pd.read_csv("../datasets/data/movies_features.txt")
new_temp_data.head()

In [ ]:
np.array(new_temp_data)